In [100]:
import pandas as pd
import jalali_pandas
import numpy as np
import psycopg2
from dbconfig import NAME, USER, PASSWORD

In [101]:
conn_metafid = psycopg2.connect(
    database=NAME, user=USER, password=PASSWORD, host='localhost', port='5432'
)

In [105]:
def net_income_query(conn):
    cursor = conn.cursor()
    cursor.execute(
        "SELECT i.date_id,i.issue_date,i.period,i.symbol_id, t.symbol_far,i.net_income,i.listed_capital FROM fundamentals_incomestatement i left join tsedata_ticker t on t.symbol = i.symbol_id where t.activity_id=5110102011")
    net_income_query_ = cursor.fetchall()
    net_income = pd.DataFrame(np.array(net_income_query_),
                              columns=["date_id","issue_date",  "period", "symbol_id","symbol_far", "net_income","listed_capital" ])
    cursor.close()
    return net_income


net_income_df = net_income_query(conn_metafid)

In [106]:
def agg_net_income(df):
    df = df.groupby(by=["symbol_id"])
    agg_net_income_ = pd.DataFrame()
    for name, group in df:
        group["agg_net_income"] = group["net_income"].rolling(4).sum()
        agg_net_income_ = pd.concat([agg_net_income_, group])
    agg_net_income_ = agg_net_income_[agg_net_income_["agg_net_income"].notnull()]
    return agg_net_income_


agg_net_income_df = agg_net_income(net_income_df)
agg_net_income_df

/var/folders/w0/18ktbnvn1mzfbp28d98ch3x40000gn/T/ipykernel_3962/3161638710.py:4: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in df:


,date_id,issue_date,period,symbol_id,symbol_far,net_income,listed_capital,agg_net_income
145,2013-06-21,2013-07-17,1,KRSN1,خراسان,1148725,1789912,3571701.0
146,2013-09-22,2013-12-16,2,KRSN1,خراسان,1096917,1789912,3980262.0
147,2013-12-21,2014-01-06,3,KRSN1,خراسان,814465,1789912,4383986.0
148,2014-03-20,2014-05-10,4,KRSN1,خراسان,884105,1789912,3944212.0
149,2014-06-21,2014-07-22,1,KRSN1,خراسان,1113094,1789912,3908581.0
...,...,...,...,...,...,...,...,...
90,2021-09-22,2021-10-22,2,PSHZ1,شیراز,14336089,5100000,40439633.0
91,2021-12-21,2022-01-20,3,PSHZ1,شیراز,8107257,5100000,36068735.0
92,2022-03-20,2022-06-27,4,PSHZ1,شیراز,27782072,5100000,56727528.0
93,2022-06-21,2022-07-20,1,PSHZ1,شیراز,23647029,5100000,73872447.0


In [104]:
def shares_hist_query(conn):
    cursor = conn.cursor()
    cursor.execute(
        "SELECT s.date_id,i.symbol_id,i.issue_date,i.period,i.net_income , t.symbol_far FROM fundamentals_shareshist s left join tsedata_ticker t on t.symbol = i.symbol_id where t.activity_id=5110102011")
    net_income_query_ = cursor.fetchall()
    net_income = pd.DataFrame(np.array(net_income_query_),
                              columns=["date_id", "symbol_id", "issue_date", "period", "net_income", "symbol_far"])
    cursor.close()
    return net_income


net_income_df = net_income_query(conn_metafid)